In [329]:
# Date: Jan 5, 2022
# Author: Sashka Warner
# Desc: 
# -- Fetches historical air quality data from OpenWeather API,
# -- Fills gaps in air quality data based on averages for a given timestep, 
# -- Summarizes data by daily maximum values
# -- Creates and trains a Long Short-Term Memory (LSTM) model to predict future air quality

In [330]:
# Imports
import os
import re
from dotenv import load_dotenv
from datetime import datetime, timedelta
import collections
import time
import numpy as np
import pandas as pd
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import joblib

In [331]:
# Load credentials
load_dotenv()
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

# Create basic URL for API call
WEATHER_API_URL = 'https://api.openweathermap.org/data/2.5/air_pollution/history'

In [332]:
def get_data(lat, long, date1, date2):
    '''
    Submits request to API for air quality data filtered based on provided lat, long, start date, and end date.
    Note: Can request max 1000 API calls / day without incurring overages.

    Parameters:
    ------------
    lat: Latitude (decimal degrees)
    long: Longitude (decimal degrees)
    date1: Start date (unix time, UTC time zone), e.g. start=1606488670
    date2: End date (unix time, UTC time zone), e.g. end=1606747870

    Returns:
    ------------
    1) json data from API on successful request, OR
    2) -1 if unsuccessful request

    '''
    # Create url from params
    params = {
        'lat': lat,
        'lon': long,
        'start': date1,
        'end': date2,
        'appid': WEATHER_API_KEY,
    }
    
    try:
        response = requests.get(url=WEATHER_API_URL, params=params)
        return response.json()['list']
    except:
        return -1

In [333]:
# Specify position
site_lat = 44.026280
site_long = -123.083715

# Specify time range for filtering
d1 = datetime(2017, 1, 1)
d2 = datetime(2022, 12, 31)
# Convert to unix time for api call
d1_unix = int(time.mktime(d1.timetuple()))
d2_unix = int(time.mktime(d2.timetuple()))

# Get air quality data formatted as json based on 
# query parameters
air_quality_json = get_data(
    lat=site_lat, 
    long=-site_long, 
    date1=d1_unix, 
    date2=d2_unix)

In [337]:
# Convert air quality data to DataFrame
air_quality = pd.json_normalize(air_quality_json)
# Format time stamps
air_quality['dt'] = pd.to_datetime(air_quality['dt'], unit='s')
air_quality.head()

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3
0,2020-11-25 01:00:00,1,236.99,0.60,3.56,66.52,1.97,7.08,8.78,17.73
1,2020-11-25 02:00:00,1,233.65,0.92,2.87,65.80,1.91,6.16,7.67,16.97
2,2020-11-25 03:00:00,1,223.64,0.91,2.16,63.66,1.62,4.53,5.74,13.81
3,2020-11-25 04:00:00,1,213.62,0.64,1.54,67.23,2.71,2.81,3.43,6.02
4,2020-11-25 05:00:00,1,210.29,0.60,1.56,69.38,3.58,2.30,2.70,3.33


In [338]:
# Remove dots from column names
air_quality.columns = air_quality.columns.map(lambda x: re.sub('\.', '_', x))
air_quality.reset_index(drop=True, inplace=True)
air_quality.head()

,dt,main_aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3
0,2020-11-25 01:00:00,1,236.99,0.60,3.56,66.52,1.97,7.08,8.78,17.73
1,2020-11-25 02:00:00,1,233.65,0.92,2.87,65.80,1.91,6.16,7.67,16.97
2,2020-11-25 03:00:00,1,223.64,0.91,2.16,63.66,1.62,4.53,5.74,13.81
3,2020-11-25 04:00:00,1,213.62,0.64,1.54,67.23,2.71,2.81,3.43,6.02
4,2020-11-25 05:00:00,1,210.29,0.60,1.56,69.38,3.58,2.30,2.70,3.33


In [339]:
# Check types and NAs
for i in air_quality.columns:
    print(i, 'type:', air_quality[i].map(lambda x: type(x)).unique(), 'Has NAs:', any(air_quality[i].isna()))

dt type: [<class 'pandas._libs.tslibs.timestamps.Timestamp'>] Has NAs: False
main_aqi type: [<class 'int'>] Has NAs: False
components_co type: [<class 'float'>] Has NAs: False
components_no type: [<class 'float'>] Has NAs: False
components_no2 type: [<class 'float'>] Has NAs: False
components_o3 type: [<class 'float'>] Has NAs: False
components_so2 type: [<class 'float'>] Has NAs: False
components_pm2_5 type: [<class 'float'>] Has NAs: False
components_pm10 type: [<class 'float'>] Has NAs: False
components_nh3 type: [<class 'float'>] Has NAs: False


In [340]:
# Examine gaps in time series
air_quality.at[1,'dt'] - air_quality.at[0, 'dt']

# Extract date time column
aq_dates = air_quality['dt']

# Create dict to store unique time deltas
unique_deltas = collections.defaultdict(int)

# Count instances of unique time deltas
for i, val in enumerate(aq_dates):
    if(i > 0):
        t_delta = aq_dates[i] - aq_dates[i-1]
        unique_deltas[t_delta] += 1
        #Examine outliers and their PM2.5 values
        if(t_delta != timedelta(hours=1)):
            print(
                't-1:', aq_dates[i-1], 'PM2.5:', air_quality.at[i-1, 'components_pm2_5'], 
                't:', aq_dates[i], 'PM2.5:', air_quality.at[i, 'components_pm2_5'])
    
unique_deltas

t-1: 2021-01-27 00:00:00 PM2.5: 3.64 t: 2021-01-28 01:00:00 PM2.5: 2.13
t-1: 2022-01-23 00:00:00 PM2.5: 51.82 t: 2022-01-24 01:00:00 PM2.5: 32.85
t-1: 2022-02-20 00:00:00 PM2.5: 1.46 t: 2022-02-21 01:00:00 PM2.5: 1.87
t-1: 2022-07-20 00:00:00 PM2.5: 9.06 t: 2022-07-22 01:00:00 PM2.5: 35.14
t-1: 2022-12-09 00:00:00 PM2.5: 6.05 t: 2022-12-11 01:00:00 PM2.5: 4.42
t-1: 2022-12-25 00:00:00 PM2.5: 3.42 t: 2022-12-26 01:00:00 PM2.5: 18.55


defaultdict(int,
            {Timedelta('0 days 01:00:00'): 18193,
             Timedelta('1 days 01:00:00'): 4,
             Timedelta('2 days 01:00:00'): 2})

In [341]:
# Create list for filled time steps
fill_steps = []

# Define variable for 1 hr difference
one_hr = timedelta(hours=1)

# Add time steps for gaps in time series
for i, val in enumerate(aq_dates):
    if(i > 0):
        # Create counter for while loop
        # if we need to gap fill the timeseries
        loop_counter = i - 1
        # Get [i-1] timestep
        last_timestep = aq_dates[i-1]
        # Get [i] timestep
        current_timestep = aq_dates[i]
        # Calculate time delta
        t_delta = current_timestep - last_timestep
        
        while(t_delta > one_hr):
            # Add one hour to the [i-1] timestep
            new_timestep = last_timestep + one_hr
            #print(f'Current timestep: {last_timestep}, Adding timestep: {new_timestep}')
            # Add the new timestep to the list
            fill_steps.append(new_timestep)
            # Calculate time delta from [i] timestep
            t_delta = aq_dates[i] - new_timestep
            #print(f't_delta: {t_delta > one_hr}')
            # Increment the counter
            loop_counter += 1
            # Store the current timestep so we can add one hour to 
            # this value if needed
            last_timestep = new_timestep

len(fill_steps)

# Check that no duplicates in time steps
#duplicates = False
#for i in collections.Counter(fill_steps):
#    if(collections.Counter(fill_steps)[i] > 1):
#        duplicates = True

#print(duplicates)
#list(map(lambda x: x if x[0] else '', collections.Counter(fill_steps)))

192

In [342]:
# Check that no duplicates in source data
#dup = False
#count_dt = collections.Counter(air_quality['dt'])
#for i in count_dt:
#    if(count_dt[i] > 1):
#        dup = True

#print(dup)

In [343]:
# Create dataframe from filled timesteps
fill_df = pd.DataFrame(data=fill_steps, columns=['dt'])

# Join new timestamps to dataframe
aq_join_fill = air_quality.merge(right=fill_df, on='dt', how='outer', suffixes=['', '_join'])

# Examine missing data after join
aq_join_fill[aq_join_fill['components_pm2_5'].isna()].head()

,dt,main_aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3
18200,2021-01-27 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18201,2021-01-27 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18202,2021-01-27 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18203,2021-01-27 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18204,2021-01-27 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [344]:
# Check that no duplicates in first join
#dup = False
#count_dt = collections.Counter(aq_join_fill['dt'])
#for i in count_dt:
#    if(count_dt[i] > 1):
#        dup = True

#print(dup)

In [345]:
# Create copy of dt col before setting dt index so we don't lose time data
# Create DateTime index
aq_join_fill['DateTime'] = aq_join_fill['dt']
aq_join_fill_dt = aq_join_fill.set_index('dt')
#print(aq_join_fill.head())

# Average values for each day & hour to fill in missing values in the time series
averages = aq_join_fill_dt.groupby(
    [
        aq_join_fill_dt.index.strftime('%m'), # Pad dates to avoid duplicates
        aq_join_fill_dt.index.strftime('%d'), 
        aq_join_fill_dt.index.strftime('%H')]).mean()
        
#averages.shape
#print(averages.head())
#Check if PM2.5 data has nulls after averaging
print('NAs in PM2.5 data after averaging over time: ', averages['components_pm2_5'].isna().sum())

NAs in PM2.5 data after averaging over time:  0


/var/folders/bx/8_p36bpd14nckr66rq2kk6kc0000gn/T/ipykernel_6412/1078082750.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aq_join_fill_dt.index.strftime('%H')]).mean()


In [346]:
# Prep tables for join
averages['m_d_h'] = averages.index.map(lambda x: str(x[0]) + str(x[1]) + str(x[2]))
#any(averages.duplicated(subset=['m_d_h']))
aq_join_fill_dt['m_d_h'] = aq_join_fill_dt.index.map(lambda x: x.strftime('%m') + x.strftime('%d') + x.strftime('%H'))
averages.head()
#aq_join_fill_dt.head()

main_aqi  components_co  components_no  components_no2  \
dt dt dt                                                           
01 01 00       1.5        363.825          0.010          12.040   
      01       1.5        367.165          0.485          11.225   
      02       2.0        365.495          1.380          10.455   
      03       2.0        378.850          2.245          12.210   
      04       2.0        437.260          5.655          15.060   

          components_o3  components_so2  components_pm2_5  components_pm10  \
dt dt dt                                                                     
01 01 00         52.570           6.695            11.065           14.460   
      01         53.640           6.500            11.760           15.430   
      02         55.075           7.315            11.615           15.565   
      03         54.005          11.965            12.535           17.165   
      04         61.870          28.610            13.110           18.350   

          components_nh3   m_d_h  
dt dt dt                          
01 01 00          16.020  010100  
      01          16.025  010101  
      02          15.960  010102  
      03          15.830  010103  
      04          10.260  010104

In [347]:
# Join averages to time series
#print(aq_join_fill_dt.shape)
aq_averaged = aq_join_fill_dt.merge(right=averages, how='left', on = 'm_d_h', suffixes= ('', '_avg'))
#aq_averaged.reset_index(drop=True, inplace=True)
# Check duplicates
print(f'Has duplicate dates? {any(aq_averaged["DateTime"].duplicated())}')

# Check which dates have missing data
missing_data = aq_averaged[aq_averaged['components_co'].isna()]['DateTime'].to_list() 
print(f'Null dates match initial gap filling? {missing_data == fill_steps}\n# Missing records: {len(missing_data)}')

Has duplicate dates? False
Null dates match initial gap filling? True
# Missing records: 192


In [348]:
# Compile original data column names
data_cols = [
    'main_aqi',
    'components_co', 
    'components_no', 
    'components_no2',
    'components_o3',
    'components_so2',
    'components_pm2_5',
    'components_pm10',
    'components_nh3']

# Update missing values with joined data
for col_name in data_cols:
    aq_averaged[col_name] = aq_averaged[col_name].where(aq_averaged[col_name].notnull(), aq_averaged[col_name + '_avg'])


# Check that all NAs have been removed from PM2pt5 data
print(f'Number of missing records: {aq_averaged["components_pm2_5"].isna().sum()}')

Number of missing records: 0


In [349]:
# Extract only columns of interest
data_cols.insert(0, 'DateTime')
aq_averaged = aq_averaged[data_cols]
aq_averaged.head()

,DateTime,main_aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3
0,2020-11-25 01:00:00,1.0,236.99,0.60,3.56,66.52,1.97,7.08,8.78,17.73
1,2020-11-25 02:00:00,1.0,233.65,0.92,2.87,65.80,1.91,6.16,7.67,16.97
2,2020-11-25 03:00:00,1.0,223.64,0.91,2.16,63.66,1.62,4.53,5.74,13.81
3,2020-11-25 04:00:00,1.0,213.62,0.64,1.54,67.23,2.71,2.81,3.43,6.02
4,2020-11-25 05:00:00,1.0,210.29,0.60,1.56,69.38,3.58,2.30,2.70,3.33


In [350]:
# Now take the daily maximum value
# Create DateTime index
aq_daily_max = aq_averaged.copy()
# Create copy so we don't lose the column
aq_daily_max['DateTimeCopy'] = aq_daily_max['DateTime']
aq_daily_max= aq_daily_max.set_index("DateTimeCopy")

# Calculate daily maximum values 
aq_daily_max = aq_daily_max.groupby(aq_daily_max.index.strftime('%Y-%m-%d')).max()
aq_daily_max.head()

,DateTime,main_aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3
DateTimeCopy,,,,,,,,,,
2020-11-25,2020-11-25 23:00:00,1.0,257.02,0.92,8.31,70.10,4.83,7.08,8.78,17.73
2020-11-26,2020-11-26 23:00:00,2.0,323.77,2.82,11.48,50.07,5.78,10.12,13.93,21.28
2020-11-27,2020-11-27 23:00:00,1.0,250.34,1.01,4.33,71.53,2.06,3.90,5.41,15.71
2020-11-28,2020-11-28 23:00:00,2.0,407.22,0.72,30.50,72.96,6.26,16.58,21.24,20.52
2020-11-29,2020-11-29 23:00:00,1.0,250.34,1.01,8.23,76.53,1.83,4.27,6.69,14.95


In [351]:
# Split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# define input sequence
#raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
raw_seq = aq_daily_max['components_pm2_5']
# choose a number of time steps
n_steps_in, n_steps_out = 3, 3
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[ 7.08 10.12  3.9 ] [16.58  4.27  4.69]
[10.12  3.9  16.58] [ 4.27  4.69 20.72]
[ 3.9  16.58  4.27] [ 4.69 20.72  4.15]
[16.58  4.27  4.69] [20.72  4.15  2.22]
[ 4.27  4.69 20.72] [ 4.15  2.22 11.03]
[ 4.69 20.72  4.15] [ 2.22 11.03 14.42]
[20.72  4.15  2.22] [11.03 14.42  9.52]
[ 4.15  2.22 11.03] [14.42  9.52  3.21]
[ 2.22 11.03 14.42] [ 9.52  3.21 22.49]
[11.03 14.42  9.52] [ 3.21 22.49 28.8 ]
[14.42  9.52  3.21] [22.49 28.8  16.99]
[ 9.52  3.21 22.49] [28.8  16.99 13.02]
[ 3.21 22.49 28.8 ] [16.99 13.02  8.47]
[22.49 28.8  16.99] [13.02  8.47  3.8 ]
[28.8  16.99 13.02] [8.47 3.8  3.32]
[16.99 13.02  8.47] [ 3.8   3.32 14.35]
[13.02  8.47  3.8 ] [ 3.32 14.35 19.53]
[8.47 3.8  3.32] [14.35 19.53 16.73]
[ 3.8   3.32 14.35] [19.53 16.73  2.26]
[ 3.32 14.35 19.53] [16.73  2.26 10.71]
[14.35 19.53 16.73] [ 2.26 10.71 10.3 ]
[19.53 16.73  2.26] [10.71 10.3  25.5 ]
[16.73  2.26 10.71] [10.3  25.5  66.03]
[ 2.26 10.71 10.3 ] [25.5  66.03 59.5 ]
[10.71 10.3  25.5 ] [66.03 59.5   7.15]
[10.3 

In [352]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X)

[[[ 7.08 ]
  [10.12 ]
  [ 3.9  ]]

 [[10.12 ]
  [ 3.9  ]
  [16.58 ]]

 [[ 3.9  ]
  [16.58 ]
  [ 4.27 ]]

 ...

 [[ 4.33 ]
  [13.465]
  [33.22 ]]

 [[13.465]
  [33.22 ]
  [36.27 ]]

 [[33.22 ]
  [36.27 ]
  [ 7.43 ]]]


In [353]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0)

In [354]:
# demonstrate prediction
#x_input = np.array([70, 80, 90])
x_input = np.array(aq_daily_max['components_pm2_5'].iloc[0:3])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[13.28305  32.633476 39.781086]]


In [355]:
# Save the model
#joblib_file = "LSTM_model.joblib"
#joblib.dump(model, joblib_file) 